In [20]:
import pandas as pd
import numpy as np
import itertools
from itertools import permutations
import time
import configparser
import netCDF4 as nc
import os
import requests

In [21]:
#Download surface data and domain data if not available on machine
def get_domain_surf_data(domain_URL,surf_URL,dirc,outFolder):
    dirc = 'C:/Users/zahu573/D/ELM/'
    outDir = os.path.join(dirc,outFolder)
    try: 
        os.makedirs(outDir)
        print(outDir)
    except:
        print(outDir,': path already exists')

    for url in [domain_URL,surf_URL]:
        if os.path.isfile(outDir+'/'+url.split('/')[-1])==True:
            print('File exists')
        else:
            r = requests.get(url, verify=False)  
            with open(outDir+'/'+url.split('/')[-1], 'wb') as f:
                print(url.split('/')[-1])
                f.write(r.content)
    clm_gridded_domain_filename = outDir+'/'+domain_URL.split('/')[-1]
    clm_gridded_surfdata_filename = outDir+'/'+surf_URL.split('/')[-1]
    print('domain file: ',clm_gridded_domain_filename)
    print('surface file: ',clm_gridded_surfdata_filename)
        
    return clm_gridded_domain_filename,clm_gridded_surfdata_filename

In [22]:
#lat_cmax: maximum lat of grid centroid
#lat_cmin: minimum lat of grid centroid
#lon_cmax: maximum lon of grid centroid
#lon_cmin: minimum lon of grid centroid
#generate the lat/lons of grid centroids in the study domain
def create_lat_lon_list(lat_cmax,lat_cmin,lon_cmax,lon_cmin,res):
    lat = lat_cmin
    lon = lon_cmin
    list_lat = [lat]
    list_lon = [lon]

    while lat<lat_cmax:
        lat+=res
        # print(lat)
        list_lat.append(lat)
    while lon<lon_cmax:
        lon+=res
        # print(lon)
        list_lon.append(lon)
    print(len(list_lat),len(list_lon),len(list_lat)*len(list_lon))
    #make unique combinations of grid centroid lat/lons
    permut = list(itertools.product(list_lat,list_lon))
    lat_lon_df = pd.DataFrame(permut,columns=['lat','lon'])
    nsites = int(len(lat_lon_df))
 
    nsites_line = pd.DataFrame({'lat': nsites, 'lon': ''}, index=[0])
    output_latlon_df_nsites = pd.concat([nsites_line,lat_lon_df]).reset_index(drop=True)
    output_latlon_df = lat_lon_df
    output_latlon_df_nsites.to_csv('C:/Users/zahu573/D/ELM/ELM_grids/%sx1_sparse_grid_latlon.txt'%str(nsites),
                            header=None,index=False, sep="\t",float_format='%.3f' )
    lat=output_latlon_df.lat.to_numpy()
    lon=output_latlon_df.lon.to_numpy()
    return lat,lon,output_latlon_df


In [23]:
def ComputeLatLonAtVertex(output_latlon_df,dlat,dlon):
    latv = np.empty((len(output_latlon_df), 4))
    lonv = np.empty((len(output_latlon_df), 4))
    for i in range(0,len(output_latlon_df)):
        latv[i][0] = output_latlon_df.lat[i]-dlat/2
        latv[i][1] = output_latlon_df.lat[i]-dlat/2 
        latv[i][2] = output_latlon_df.lat[i]+dlat/2 
        latv[i][3] = output_latlon_df.lat[i]+dlat/2
    for i in range(0,len(output_latlon_df)):
        lonv[i][0] = output_latlon_df.lon[i]-dlon/2
        lonv[i][1] = output_latlon_df.lon[i]+dlon/2 
        lonv[i][2] = output_latlon_df.lon[i]+dlon/2 
        lonv[i][3] = output_latlon_df.lon[i]-dlon/2
    return latv,lonv

In [24]:
def PerformFractionCoverCheck(varname, data, set_natural_veg_frac_to_one):
    if varname == 'PCT_URBAN':
        if set_natural_veg_frac_to_one ==1:
            data = data * 0
        else: 
            print('set_natural_veg_frac_to_one = 0')
    elif varname in ['PCT_CROP','PCT_WETLAND','PCT_LAKE','PCT_GLACIER']:
        if set_natural_veg_frac_to_one ==1:
            data = data * 0
        else:
            print('set_natural_veg_frac_to_one = 0')
    elif varname == 'PCT_NATVEG':
        if set_natural_veg_frac_to_one ==1:
            data = data*0 + 100
        else:
            print('set_natural_veg_frac_to_one = 0')
    return data

In [25]:
# Creates an unstructured domain netCDF file for CLM45.

# INPUT:
#       lon_region = Vector containing longitude @ cell-center.
#       lat_region = Vector containing latitude @ cell-center.
#       lonv_region = Vector containing longitude @ cell-vertex.
#       latv_region = Vector containing latitude @ cell-vertex.
#       clm_gridded_domain_filename = Default CLM domain file name
#       out_netcdf_dir = Directory where CLM surface dataset will be saved
#       clm_usrdat_name = User defined name for CLM dataset

def CreateCLMUgridDomainForCLM45(lat_region,lon_region,latv_region,lonv_region,
                                clm_gridded_domain_filename, out_netcdf_dir,clm_usrdat_name):
    nc_out_name = '%s/domain_%s_%s.nc'%(out_netcdf_dir,clm_usrdat_name,time.strftime("%y%m%d"))
    print('domain:  ',nc_out_name)
    
    ncid_inp = nc.Dataset(clm_gridded_domain_filename,'r')
    ncid_out = nc.Dataset(nc_out_name, 'w', format='NETCDF4')
    
    ndims=len(ncid_inp.dimensions)
    nvars=len(ncid_inp.variables)
    nv = np.shape(latv_region)[1]
    ni = np.shape(latv_region)[0]
    nj = 1
    
    '''Define dimensions'''
    for ii in ncid_inp.dimensions:
        dimname = ncid_inp.dimensions[ii].name
        ndim = len(ncid_inp.dimensions[ii])
        # print(dimname,ndim)
        if dimname == 'nv':
            ndim = nv
            nv = ncid_out.createDimension(dimname, ndim)
        elif ii == 'ni':
            ndim = ni    
            ni = ncid_out.createDimension(dimname, ndim)
        elif ii == 'nj':
            ndim = nj
            nj = ncid_out.createDimension(dimname, ndim)
            
    '''Define variables'''
    varnames = []
    varids = [0]*nvars
    for (varid,ivar) in zip(range(len(varids)),ncid_inp.variables):
        varname = ncid_inp.variables[ivar].name
        xtype = ncid_inp.variables[ivar].dtype
        dimids = ncid_inp.variables[ivar].dimensions
        natts = len(ncid_inp.variables[ivar].ncattrs())
        varnames.append(varname)
        # print(varname)
        varids[varid] = ncid_out.createVariable(varname,xtype,dimids)
        for iatt in ncid_inp.variables[ivar].ncattrs():
            attname = iatt
            attvalue = getattr(ncid_inp.variables[ivar], iatt)
            # print(attname, attvalue)
            ncid_out.variables[ivar].setncattr(attname, attvalue)    
    # print(ncid_out.variables)
    
    '''Set the global atrributes for output netcdf'''
    setattr(ncid_out, 'Created_by', os.getlogin())
    setattr(ncid_out, 'Created_on', time.strftime("%Y-%m-%d %H:%M:%S")) 
    
    '''Copy variables'''
    for ivar in ncid_inp.variables:
        varname = ncid_inp.variables[ivar].name
        xtype = ncid_inp.variables[ivar].dtype
        dimids = ncid_inp.variables[ivar].dimensions
        natts = len(ncid_inp.variables[ivar].ncattrs())
        if varname == 'xc':
            data = lon_region
            ncid_out['xc'][:] = data
        elif varname == 'yc':
            data = lat_region
            ncid_out['yc'][:] = data
        elif varname == 'xv':
            data = lonv_region
            ncid_out['xv'][:] = data
        elif varname == 'yv':
            data = latv_region
            ncid_out['yv'][:] = data
        elif varname == 'mask':
            data = np.ones((len(lon_region),1))
            ncid_out['mask'][:] = data
        elif varname == 'frac':
            data = np.ones((len(lon_region),1))
            ncid_out['frac'][:] = data
        elif varname == 'area':
            if  lonv_region.shape[1] == 3:
                ax = lonv_region[:,0]
                ay = latv_region[:,0]
                bx = lonv_region[:,1]
                by = latv_region[:,1]
                cx = lonv_region[:,2]
                cy = latv_region[:,2]
                data = 0.5*((ax*(by-cy)) + (bx*(cy-ay)) + (cx*(ay-by)))
            elif lonv_region.shape[1] == 4:
                data = (lonv_region[:,0]-lonv_region[:,1])*(latv_region[:,0]-latv_region[:,2])
                # print(data)
            else:
                print("Error: Added area computation")
            ncid_out['area'][:] = data

    ncid_out.close()
    ncid_inp.close()

In [26]:
'''Converts a 2D data (lat,lon) to 1D (gridcell)'''
def sgrid_convert_2d_to_1d(vardimids, ii_idx, jj_idx, data):
    data_2d = np.zeros(np.shape(ii_idx))
    for ii in range(np.shape(ii_idx)[0]):
        data_2d[ii]= data[ii_idx[ii],jj_idx[ii]]
    # (lon,lat) --> (gridcell)
    vardimids_new =  []
    for x in vardimids:
        if x in ['lsmlat','lsmlon']:
            if x == 'lsmlat':
                vardimids_new.append('gridcell')
        else:
            vardimids_new.append(x)
    vardimids = tuple(vardimids_new)
    dims = np.shape(data_2d)
    if len(dims)>1:
        dims_new = [dims[0]*dims[1]]+[dims[k] for k in range(2,len(dims))]
        print("length of data_2d dim exceeds 1. Update the code")
    else:
        dims_new = dims
    data_1d = np.reshape(data_2d,dims_new)
    return data_1d

In [27]:
''' Converts a 2D data (gridcell,:) to 1D (gridcell,:)'''
def ugrid_convert_2d_to_2d(ii_idx, data):
    data_2d = data[:,ii_idx.astype(int)]
    return data_2d
    

In [28]:
'''Converts a 3D data (lat,lon,:) to 2D (gridcell,:)'''
def sgrid_convert_3d_to_2d(vardimids, ii_idx, jj_idx, data):
    nx = np.shape(ii_idx)[0]
    try:
        ny = np.shape(jj_idx)[1]
    except:
        ny = 1
    try:
        nz = np.shape(data)[0] 
    except:
        nz = 1
    # print(nx,ny,nz)
    data_3d = np.zeros((nz,ny,nx))
    # print(data_3d)
    # print(np.shape(data_3d))
    for ii in range(0,nx):
        for jj in range(0,ny):
            data_3d[:,0,ii] = data[:,ii_idx[ii],jj_idx[ii]]
    vardimids_new =  []
    for x in vardimids:
        if x in ['lsmlat','lsmlon']:
            if x == 'lsmlat':
                vardimids_new.append('gridcell')
        else:
            vardimids_new.append(x)
    vardimids = tuple(vardimids_new)
    # print(vardimids)
    dims = np.shape(data_3d)
    # print(dims)
    if len(dims)>2:
        dims_new = [dims[k] for k in range(0,len(dims)-2)]+[dims[1]*dims[2]]
        # print(dims_new)
    else:
        dims_new = [dims[1]*dims[2]]
    data_2d = np.reshape(data_3d,dims_new)
    
    return data_2d 

In [29]:
'''Converts a 3D data (gridcell,:,:) to 3D (gridcell,:,:)'''
def ugrid_convert_3d_to_3d(ii_idx, data):
    nx = np.shape(ii_idx)[0]
    ny = np.shape(data)[1]
    nz = np.shape(data)[0]
    data_3d = np.zeros((nz,ny,nx))
    for ii in range(0,nx):
        data_3d[:,:,ii] = data[:,:,jj_idx[ii]]
    return data_3d

In [30]:
'''Converts a 4D data (lat,lon,:,:) to 3D (gridcell,:,:)'''
def sgrid_convert_4d_to_3d(vardimids, ii_idx, jj_idx, data):
    nx = np.shape(ii_idx)[0]
    try:
        ny = np.shape(ii_idx)[1]
    except:
        ny = 1
    nz = np.shape(data)[1]
    na = np.shape(data)[0]
    # print(nx, ny, nz,na)
    data_4d = np.zeros((na,nz,ny,nx))
    for ii in range(0,nx):
        for jj in range(0,ny):
            data_4d[:,:,0,ii] = data[:,:,ii_idx[ii],jj_idx[ii]]
    vardimids_new =  []
    for x in vardimids:
        if x in ['lsmlat','lsmlon']:
            if x == 'lsmlat':
                vardimids_new.append('gridcell')
        else:
            vardimids_new.append(x)
    vardimids = tuple(vardimids_new)
    dims = np.shape(data_4d)
    # print(vardimids,dims)
    if len(dims)>2:
        dims_new = [dims[k] for k in range(0,len(dims)-2)]+[dims[-1]*dims[-2]]
        # print(dims_new)
    else:
        dims_new = [dims[-1]*dims[-2]]
    data_3d = np.reshape(data_4d,dims_new)
    return data_3d

In [31]:
# Creates an unstructured surface-data netCDF file for CLM45.

# INPUT:
#        lati_region = Vector containing latitude @ cell-center.
#        long_region = Vector containing longitude @ cell-center.
#        clm_gridded_surfdata_filename = Gridded CLM surface data file
#        out_netcdf_dir = Directory where CLM surface dataset will be saved
#        clm_usrdat_name = User defined name for CLM dataset
#        set_natural_veg_frac_to_one =

def CreateCLMUgridSurfdatForCLM45(lati_region, long_region, clm_gridded_surfdata_filename,
                                  out_netcdf_dir, clm_usrdat_name, surf_URL, set_natural_veg_frac_to_one, dlat,dlon):
    nc_out_name = '%s/surfdata_%s_%s.nc'%(out_netcdf_dir,clm_usrdat_name,time.strftime("%y%m%d"))
    print('surface dataset:  ', nc_out_name)

    ncid_inp = nc.Dataset(clm_gridded_surfdata_filename,'r')
    ncid_out = nc.Dataset(nc_out_name, 'w', format='NETCDF3_64BIT_OFFSET')   

    ndims=len(ncid_inp.dimensions)
    nvars=len(ncid_inp.variables)
    
    '''Define dimensions'''
    lonlat_found = 0
    dimname_list=[0]*ndims
    dim_list=[0]*ndims

    # print(dimname_list)
    for (idim,n) in zip(ncid_inp.dimensions,range(ndims)):
        # print(idim,n)
        dimname = ncid_inp.dimensions[idim].name
        dimname_list[n] = dimname
        dimlen = len(ncid_inp.dimensions[idim])
        if dimname in ['lsmlon','lsmlat']:
            if lonlat_found ==0:
                lonlat_found = 1
                dimname = 'gridcell'
                dimname_list.append(dimname)
                dimlen = len(lon_region)
                # print(dimname,dimlen)
                dim_list[n] = ncid_out.createDimension(dimname, dimlen)
        elif dimname == 'time':
            # print(dimname,dimlen)
            dimname_list.append(dimname)
            dim_list[n] =  ncid_out.createDimension(dimname,None);
        elif dimname == 'gridcell':
            dimlen = len(lon_region)
            dimname_list.append(dimname)
            # print('elif',dimname,dimlen)
            dim_list[n] = ncid_out.createDimension(dimname, dimlen)
        else:
            # print('else',dimname,dimlen)
            dim_list[n] = ncid_out.createDimension(dimname, dimlen)

    '''Define variables'''
    varnames = []
    varids = [0]*nvars
    for (varid,ivar) in zip(range(nvars),ncid_inp.variables):
        varname = ncid_inp.variables[ivar].name
        xtype = ncid_inp.variables[ivar].dtype
        dimids = ncid_inp.variables[ivar].dimensions
        natts = len(ncid_inp.variables[ivar].ncattrs())
        if len(dimids)>0:
            if lonlat_found==1:
                if all(x in dimids for x in ['lsmlat', 'lsmlon']):
                    # dimids_tup2lst = list(dimids)
                    dimids_new=[]
                    for x in dimids:
                        if x in ['lsmlat','lsmlon']:
                            if x == 'lsmlat':
                                dimids_new.append('gridcell')
                        else:
                            dimids_new.append(x)
                    dimids = tuple(dimids_new)
        # print(dimids)
        varnames.append(varname)
        # print(varname)
        varids[varid] = ncid_out.createVariable(varname,xtype,dimids)
        for iatt in ncid_inp.variables[ivar].ncattrs():
            attname = iatt
            attvalue = getattr(ncid_inp.variables[ivar], iatt)
            # print(attname, attvalue)
            ncid_out.variables[ivar].setncattr(attname, attvalue)    
    # print(ncid_out.variables)
    
    '''Set the global atrributes for output netcdf'''
    setattr(ncid_out, 'Created_by', os.getlogin())
    setattr(ncid_out, 'Created_on', time.strftime("%Y-%m-%d %H:%M:%S"))
    
    '''Find the nearest neighbor index for (long_region,lati_xy) within global dataset'''
    #Get Lat/Lon for global 2D grid.
    for ivar in varnames:
        if ivar=='LATIXY':
            latixy = ncid_inp.variables[ivar][:]
        elif ivar=='LONGXY':
            longxy = ncid_inp.variables[ivar][:]

    #read in global pft mask 1=valid 0=invalid
    pftmask = ncid_inp.variables['PFTDATA_MASK'][:]    

    #mark invalid gridcells as [lon, lat] [-9999, -9999]
    latixy[pftmask==0]=-9999
    longxy[pftmask==0]=-9999

    #allocate memory
    ii_idx = np.zeros_like(lon_region)  
    jj_idx = np.zeros_like(lon_region)

    #find the index
    for ii in range(np.shape(lon_region)[0]):
        dist = (longxy-lon_region[ii])**2 + (latixy - lat_region[ii])**2
        # nearest_cell_i_idx, nearest_cell_j_idx = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
        nearest_cell_i_idx, nearest_cell_j_idx = np.where((dist==dist.min()))
        # print(nearest_cell_i_idx, nearest_cell_j_idx)
        if len(nearest_cell_i_idx) > 1:
            print('  WARNING: Site with (lat,lon) = (%s,%s) has more than one cells that are equidistant.' %(str(lat_region[ii]),str(lon_region[ii])))
            print('Picking the first closest grid cell.')
        # print(dist.min())
        ii_idx[ii] = nearest_cell_i_idx[0]
        jj_idx[ii] = nearest_cell_j_idx[0]

    ii_idx = ii_idx.astype(int)
    jj_idx = jj_idx.astype(int)
    
    '''Copy variables'''
    for ivar in ncid_inp.variables:
        data = ncid_inp.variables[ivar][:]
        varname = ncid_inp.variables[ivar].name
        vartype = ncid_inp.variables[ivar].dtype
        vardimids = ncid_inp.variables[ivar].dimensions
        varnatts = len(ncid_inp.variables[ivar].ncattrs())
        if varname == 'LATIXY':
            # print('latixy')
            ncid_out['LATIXY'][:] = lat_region
        elif varname == 'LONGXY':
            # print('lonxy')
            ncid_out['LONGXY'][:] = lon_region 
        elif varname == 'AREA':
            AREA = np.empty_like(lon_region)
            AREA.fill((111.1*dlat)*(111.1*dlon))
            ncid_out['AREA'][:] = AREA 
        else:
            if len(vardimids)==0:
                print('vardimids length 0',varname)
                ncid_out[ivar][:] = data
            elif len(vardimids)==1:
                print('vardimids length 1',varname)
                if lonlat_found==1:
                    data=0
                else:
                    if vardimids == 'lsmlat':
                        # print(vardimids)
                        data = data[ii_idx.astype(int)]
                        # print(data)
                    else:
                        data = 0
                ncid_out[ivar][:] = data
            elif len(vardimids)==2:
                print('vardimids length 2',varname)
                if 'lsmlat' in vardimids:
                    if lonlat_found==1:
                        # print((varname, vardimids,np.shape(data)))
                        data_1d = sgrid_convert_2d_to_1d(vardimids, ii_idx, jj_idx, data)
                        data_new = PerformFractionCoverCheck(varname, data_1d, set_natural_veg_frac_to_one)
                    else:
                        data_2d = ugrid_convert_2d_to_2d(ii_idx, data)
                        data_new = PerformFractionCoverCheck(varname, data_2d, set_natural_veg_frac_to_one)
                    ncid_out[ivar][:] = data_new
                else:
                    # print('else case 2')
                    ncid_out[ivar][:] = data

            elif len(vardimids)==3:
                print('vardimids length 3',varname)
                if 'lsmlat' in vardimids:
                    if lonlat_found==1:
                        # print(varname, vardimids,np.shape(data))
                        data_2d  = sgrid_convert_3d_to_2d(vardimids, ii_idx, jj_idx, data)
                        data_new = PerformFractionCoverCheck(varname, data_2d, set_natural_veg_frac_to_one)
                        ncid_out[ivar][:] = data_new        
                    else:
                        # print('case 3 else')
                        data_3d  = ugrid_convert_3d_to_3d(ii_idx, data)
                        data_new = PerformFractionCoverCheck(varname, data_3d, set_natural_veg_frac_to_one)
                        ncid_out[ivar][:] = data_new 
                else:
                    # print('no latlon')
                    ncid_out[ivar][:] = data                    
            elif len(vardimids)==4:
                print('vardimids length 4',varname)
                if 'lsmlat' in vardimids:
                    if lonlat_found==1:
                        # print(varname,np.shape(data))
                        data_3d = sgrid_convert_4d_to_3d(vardimids, ii_idx, jj_idx, data)
                    else:
                        print('error')
                    ncid_out[ivar][:] = data_3d 
                else:
                    print('else vardimids length 4')
                    ncid_out[ivar][:] = data 
            else:
                print('error')
    ncid_out.close()
    ncid_inp.close()


In [36]:
print('Downloading domain and surface netcdfs if missing')
domain_URL='https://svn-ccsm-inputdata.cgd.ucar.edu/trunk/inputdata/share/domains/domain.clm/domain.lnd.fv1.9x2.5_USGS.110713.nc'
surf_URL='https://svn-ccsm-inputdata.cgd.ucar.edu/trunk/inputdata/lnd/clm2/surfdata_map/surfdata_1.9x2.5_simyr2000_c141219.nc'
''' Another url for surface data: https://web.lcrc.anl.gov/public/e3sm/inputdata/lnd/clm2/surfdata_map/'''
dirc = 'C:/Users/zahu573/D/ELM/'
outFolder = 'clm-netcdf'
clm_gridded_domain_filename,clm_gridded_surfdata_filename = get_domain_surf_data(domain_URL,surf_URL,dirc,outFolder)

print('Calculating Lat/lon for grid center')
#setting the max and min lat/lons and resolution 
#Upper Wenas
# lat_cmax = 47.05416667
# lat_cmin = 46.7375
# lon_cmax = 239.35416667 
# lon_cmin = 239.0125

#American River
lat_cmax = 46.99583334
lat_cmin = 46.80416667
lon_cmax = 238.8458333
lon_cmin = 238.4708333

res = 0.025/3

lat_region,lon_region,output_latlon_df =create_lat_lon_list(lat_cmax,lat_cmin ,lon_cmax,lon_cmin,res)

#define grid size in degrees
dlat = 0.025/3  
dlon = 0.025/3 

#The name that will be used to for the sparse surface dataset domain file.
clm_usrdat_name = str(len(output_latlon_df))+'x1_sparse_grid'
print(clm_usrdat_name)
set_natural_veg_frac_to_one = 0

print('Calculating lat/lon vertex')
latv_region,lonv_region =ComputeLatLonAtVertex(output_latlon_df,dlat,dlon)

out_netcdf_dir=dirc+outFolder
print('Creating CLM domain')
domain_nc = CreateCLMUgridDomainForCLM45(lat_region,lon_region,latv_region,lonv_region,
                                clm_gridded_domain_filename, out_netcdf_dir,clm_usrdat_name)
print('Creating CLM surface dataset')
fsurdat = CreateCLMUgridSurfdatForCLM45(lat_region,lon_region, clm_gridded_surfdata_filename, out_netcdf_dir,
                                        clm_usrdat_name,surf_URL, set_natural_veg_frac_to_one,dlat,dlon)

C:/Users/zahu573/D/ELM/clm-netcdf : path already exists
File exists
File exists
domain file:  C:/Users/zahu573/D/ELM/clm-netcdf/domain.lnd.fv1.9x2.5_USGS.110713.nc
surface file:  C:/Users/zahu573/D/ELM/clm-netcdf/surfdata_1.9x2.5_simyr2000_c141219.nc
Calculating Lat/lon for grid center
25 47 1175
1175x1_sparse_grid
Calculating lat/lon vertex
Creating CLM domain
domain:   C:/Users/zahu573/D/ELM/clm-netcdf/domain_1175x1_sparse_grid_231025.nc
Creating CLM surface dataset
surface dataset:   C:/Users/zahu573/D/ELM/clm-netcdf/surfdata_1175x1_sparse_grid_231025.nc
vardimids length 0 mxsoil_color
vardimids length 2 SOIL_COLOR
vardimids length 3 PCT_SAND
vardimids length 3 PCT_CLAY
vardimids length 3 ORGANIC
vardimids length 2 FMAX
vardimids length 1 natpft
vardimids length 2 LANDFRAC_PFT
vardimids length 2 PFTDATA_MASK
vardimids length 2 PCT_NATVEG
set_natural_veg_frac_to_one = 0
vardimids length 2 PCT_CROP
set_natural_veg_frac_to_one = 0
vardimids length 3 PCT_NAT_PFT
vardimids length 4 MONTH